# LLM - RAG (대규모 언어 모델 - 검색 증강 생성)

# 개요
이 데모는 GPT-4 모델을 검색 증강 생성(RAG) 설정에서 사용하는 방법을 보여줍니다. \
모델은 자연어 질문을 분석하고, 지식 기반에서 관련 정보를 검색한 후, 검색된 정보를 기반으로 답변을 제공합니다.



In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

### 자연어 질문으로 시작하기

In [ ]:
input = "2025년 한국 대선에서 어느 대통령 후보가 당선될 가능성이 높은가요? 그 이유를 설명하세요."

### 1단계: GPT-4o: 이 질문에 답하기 위해 무엇이 필요한가요?

In [3]:
system_prompt=f'''다음 질문에 가능한 한 최선을 다해 답하세요. 다음 도구에 액세스할 수 있습니다:

웹 검색: 웹을 사용하여 정보를 찾습니다.

아래 형식을 엄격하게 따라야 합니다:

질문: 반드시 답해야 하는 입력 질문
생각: 무엇을 해야 할지 항상 생각해야 합니다.
행동: 취해야 할 행동, [조사] 중 하나이어야 합니다.
행동 입력: 행동에 대한 입력

시작!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [4]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

Question: 중3 수학캠프 추천
Thought: 중학교 3학년 학생들을 위한 수학 캠프 추천 정보를 찾아야 한다.
Action: [조사]
Action input: 중3 수학 캠프 추천 2024년 기준, 서울 및 전국 중3 대상 수학 캠프 후기 및 평가 정보 검색


### 2단계: 더 많은 세부 정보를 위해 웹 검색


계속하기 전에 구독에서 검색 서비스를 생성하세요 [검색 리소스 생성](https://serpapi.com). \
이 샘플은 쿼리를 사용하여 웹 검색 API에 호출을 수행하고 관련 웹 검색 결과를 반환합니다.

In [ ]:

import serpapi
import json

params = {
  "engine": "google",
  "q": "2025년 한국 대선에서 어느 대통령 후보가 당선될 가능성이 높은가요? 그 이유를 설명하세요.",
  "api_key": os.getenv("SERP_API_KEY"),
}
print()

response = serpapi.search(params)

# response 객체를 JSON 문자열로 변환 후, ensure_ascii=False로 한글이 깨지지 않게 출력
print(json.dumps(response.data, ensure_ascii=False, indent=2))

context = ''
organic_results = response["organic_results"]

for i, result in enumerate(organic_results):
    title = result.get("title")
    link = result.get("link")
    snippet = result.get("snippet")
    # Ensure the title is printed as a proper Korean string
    if isinstance(snippet, bytes):
        snippet = snippet.decode('utf-8')
    if isinstance(title, bytes):
        title = title.decode('utf-8')
    print(f"Result {i+1}:")
    print(f"Title: {title}")
    print(f"Link: {link}")
    print(f"Snippet:{snippet}")
    context += snippet + '\n'


{
  "search_metadata": {
    "id": "6826ba0b71a90dd10a5a5832",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/54dabe157fb64c5f/6826ba0b71a90dd10a5a5832.json",
    "created_at": "2025-05-16 04:07:39 UTC",
    "processed_at": "2025-05-16 04:07:39 UTC",
    "google_url": "https://www.google.com/search?q=%EC%A4%913%ED%95%99%EB%85%84+%EC%88%98%ED%95%99+%EC%BA%A0%ED%94%84+%EC%B6%94%EC%B2%9C&oq=%EC%A4%913%ED%95%99%EB%85%84+%EC%88%98%ED%95%99+%EC%BA%A0%ED%94%84+%EC%B6%94%EC%B2%9C&sourceid=chrome&ie=UTF-8",
    "raw_html_file": "https://serpapi.com/searches/54dabe157fb64c5f/6826ba0b71a90dd10a5a5832.html",
    "total_time_taken": 6.28
  },
  "search_parameters": {
    "engine": "google",
    "q": "중3학년 수학 캠프 추천",
    "google_domain": "google.com",
    "device": "desktop"
  },
  "search_information": {
    "query_displayed": "중3학년 수학 캠프 추천",
    "total_results": 375000,
    "time_taken_displayed": 0.29,
    "organic_results_state": "Results for exact spelling"
  },
 

### 추가 처리를 위한 수집된 정보

In [7]:
print(context)

자녀가 입시 고난에 휘둘리지 않으려면 발 빠른 부모님의 현명한 선택과 판단이 요구됩니다. 그래서 이미 매년 겨울마다 아는공부캠프로 전국의 중학교 3 ...
수학 경시대회는 다른 여름 프로그램에 비해 수강 결과가 자명하기 때문에 4년동안 가장 비싼 캠프에 다니며 AMC에서 100점 받은 학생보다 집에서 혼자 ...
숨마쿰라우데 중학 수학 개념 기본서 :이과생에게는 불호일 수 있음, 개념 설명이 아주 잘 되어 있는 개념서로 체계적인 개념학습으로 실력을 쌓기 좋다.
서울대 합격생 190명이 강력 추천한 수학 올패스 수능날까지 차길영 선생님의 모든 강좌 무제한 수강 🟨올패스 보러가기 ➡️ https://bit.ly/3hpdm7M *매 ...
강의구성61강 ; 맛보기2강 ; 강의범위, 중학교 수학 3학년 2학기 전 범위 ; 강좌특징 · 개념완성과 함께 문제 적용력도 키우는 강좌 1. 단단한 개념완성 - 어떤 문제를 만나도 ...
대한민국 대표 썸머윈터스쿨 여름겨울방학캠프. 중3,고1,고2,고3 추천 기숙학원. 자기주도학습코칭센터, 나만의 전교1등 공부법 체득!
자유학기제로 변화되는 수업대비는 물론 진급 후 수학공부에 자신감을 실어줄 수 있는 강의입니다. 강의 목록. 총 48/48 + OT 개 의 강의가 완강 되었습니다. 강의 ...
팔로알토에서 1.캐스티야 2.스트랫포드 3.챌린저 이렇게 보고 있는데 혹시 위의 세 학교 써머 경험이 있으신 분 경험담과 조언 부탁드려요. 제가 아이들이 ...
대치동수학 #수학문제집추천 #개념 #심화 ———— 대치동캐슬 수학센터 수업 시간표 : https://cafe.naver.com/daechidongcastle/1353 고대원T 인스타 ...



In [8]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### 3단계: GPT-4o: 이 질문에 답하기 위해 무엇을 해야 하나요? (다시)

In [9]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

중3 수학캠프를 추천할 때는 개념 완성과 문제 적용력을 함께 키울 수 있는 곳을 선택하는 것이 좋습니다. 예를 들어, 겨울마다 진행되는 아는공부캠프는 전국 중학교 3학년 학생들에게 인기 있으며, 체계적인 개념학습과 실력 향상에 초점을 맞추고 있습니다. 또한, '숨마쿰라우데 중학 수학 개념 기본서'와 연계된 캠프도 개념 설명이 잘 되어 있어 기본기를 탄탄히 하는 데 도움이 됩니다.

대치동 캐슬 수학센터 같은 대치동 지역의 학원들도 심화 및 개념 강의를 체계적으로 제공하니 참고해 보시기 바랍니다. 

또한, 수학 경시대회 준비를 겸하는 캠프도 있는데, 수강 결과가 명확하므로 목표가 뚜렷하다면 이러한 프로그램 참여도 고려해 볼 만합니다.

마지막으로, 본인의 학습 성향과 목표에 맞는 프로그램인지, 강사진과 커리큘럼이 알차고 체계적인지 꼭 확인한 후 선택하는 것을 권장합니다.


### GPT-4o: 이제 질문에 대한 최종 답변을 얻었습니다.

#### 탐색할 사용 사례
1. **지식 관리** \
내부 문서, 매뉴얼 및 정책에서 정보를 검색하고 제공하여 기업 지식에 대한 접근성을 향상시킵니다.

2. **비즈니스 인텔리전스 및 분석** \
시장 분석가가 다양한 소스에서 관련 시장 데이터, 보고서 및 통찰력을 검색할 수 있도록 지원합니다.

3. **법률 및 준수** \
법률 전문가가 쿼리에 따라 관련 판례, 법령 및 법적 선례를 검색할 수 있도록 지원합니다.